In [41]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [42]:
feature_filename = "data/Features.csv"
label_filename = "data/Labels.csv"
df_feat = pd.read_csv(feature_filename)
df_label = pd.read_csv(label_filename)
df = pd.merge(df_feat, df_label, on='id', how='inner')

del df_feat
del df_label

# Data Cleaning

## Drop columns
for more details on why these columns are dropped please refer to the data_understanding.ipynb file

In [43]:
df.drop(["date_recorded", "scheme_name", "num_private", "wpt_name", "subvillage", "lga", "ward", "public_meeting", "recorded_by", "permit", "extraction_type","extraction_type_group", "extraction_type_class", "scheme_management", "management", "management_group","payment", "water_quality", "quality_group", "quantity_group", "source_type" ,"source_class", "waterpoint_type_group"], inplace=True, axis=1)


## Remove outliers
for more details on why I chose to remove these outliers and how I came up with the thresholds please refer to the data_understanding.ipynb file

In [44]:
df = df[df["amount_tsh"] <= 120]
df = df[df["population"] <= 10000]

## Impute missing values

In [45]:
def median_imputer(df, ref_col, feat_ls, mis_values):
    """Imputes each instance with missing value for each feature in the feat_ls list with the median value
    of instances grouped by the ref_col feature
    :argument:
        df(pd.Dataframe): pd.Dataframe containing the data
        ref_col(str):  containing the reference column/feature
        feat_ls(list): list of features that need to be imputed
        mis_values(list): list containing the placeholder values for missing values (e.g.: 0, NaN) for each feature
    """
    df_med = df.groupby([ref_col])[feat_ls].median()

    def impute(row):
        for att in feat_ls:
            row[att] = df_med.loc[row[ref_col]][att]
        return row

    for att, mv in zip(feat_ls,mis_values):
        df.loc[df[att]==mv, att] = df[df[att]==mv].apply(impute, axis=1)[att]

median_imputer(df, "district_code", ["gps_height", "population", "longitude", "latitude"], [0,0, 0,-2.000000e-08])

In [46]:
imp_median = SimpleImputer(missing_values=0, strategy='median')
imp_median.fit_transform(df[["construction_year"]])
imp_median.fit_transform(df[["amount_tsh"]])

array([[30.],
       [25.],
       [30.],
       ...,
       [30.],
       [30.],
       [30.]])

### Reduce Categories

In [47]:
df["waterpoint_type"].mask(df["waterpoint_type"].isin(["improved spring", "cattle trough", "dam"]), "other", inplace=True)
df["waterpoint_type"].mask(df["waterpoint_type"].isin(["communal standpipe multiple"]), "communal standpipe", inplace=True)
df["source"].mask(df["source"].isin(["other"]), "unknown", inplace=True)
df["payment_type"].mask(df["payment_type"].isin(["other"]), "unknown", inplace=True)

### Drop null values

In [48]:
df.dropna(inplace=True)

In [49]:
df["status_group"].value_counts()

functional                 23040
non functional             19518
functional needs repair     2929
Name: status_group, dtype: int64

### save cleaned dataset

In [50]:
df.to_csv("data/cleaned_data.csv", index=False)